In [1]:
library(Seurat)
library(monocle3)
pyenv = 'cellpath'

library(anndata)
library(reticulate)
use_condaenv(pyenv, required = TRUE)

Loading required package: SeuratObject

Loading required package: sp


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: Biobase

Warning message:
“package ‘Biobase’ was built under R version 4.4.1”
Loading required package: BiocGenerics

Warning message:
“package ‘BiocGenerics’ was built under R version 4.4.1”

Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:SeuratObject’:

    intersect


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, table,
    tappl

In [25]:
cds = readRDS('../data/CE_CB/for_qin_briggsae_elegans/cds_bg_20240903.rds')

In [ ]:
colnames(pData(cds))

[1] "barcode"                   "Size_Factor"              
 [3] "n.umi"                     "cell"                     
 [5] "sample.name"               "batch"                    
 [7] "time.point"                "species"                  
 [9] "suffix"                    "sample"                   
[11] "ws260_n.umi"               "ws260_Size_Factor"        
[13] "cell.type"                 "cell.subtype"             
[15] "plot.cell.type"            "raw.embryo.time"          
[17] "embryo.time"               "lineage_packer"           
[19] "genotype"                  "assigned_cell_type"       
[21] "to.filter"                 "cluster.cell.type"        
[23] "cell.subtype.2"            "cluster"                  
[25] "umi.quant"                 "umap_1"                   
[27] "umap_2"                    "cell.class"               
[29] "joint.cell.subtype"        "round.2.doublets"         
[31] "tissue.type"               "mt.frac"                  
[33] "round.1.cell.type"         "plot.ct"                  
[35] "n.r1ct.scores.past.thresh" "initial.cluster"          
[37] "max.score"                 "br.cell.subtype"          
[39] "to.filter2"                "cell.old.name"            
[41] "to.filter.bkgd"            "br.broad.cell.type"       
[43] "cluster_seurat"            "jt_cell_class"            
[45] "smoothed.embryo.time"      "embryo.time.bin"          
[47] "jt.cell.class.time.bin"    "jt.cell.subtype2"         
[49] "old_cds_barcodes"          "manual_lineage"           
[51] "lineage_auto_briggsae"     "manual_lineage2"          
[53] "db.version"                "batch4"                   
[55] "batch3"                    "batch_300"                
[57] "batch_400"                 "batch_500"                
[59] "batch_120"                 "batch_180"                
[61] "batch_240"                 "batch_360"                
[63] "cell_short"                "source"                   
[65] "ws260_cell"                "global_cluster"           
[67] "supp.cell.type"            "supp.cell.type2"          
[69] "supp.cell.type3"           "est.bg.prop.mus"          
[71] "est.bg.prop.hyp"           "est.bg.prop.both"         
[73] "est.bg.prop"               "dataset"                  
[75] "dataset2"                  "dataset3"                 
[77] "cell_type"                 "filter"                   
[79] "cel_lineage_specific"      "cbr_lineage_specific"     
[81] "cel_lineage_broad"         "cbr_lineage_broad"        
[83] "lineage_broad"             "lineage_specific"         
[85] "cel_lineage_complete"      "cbr_lineage_complete"     
[87] "lineage_complete"          "germline_pseudotime"      
[89] "global_local_cluster"

In [5]:
seurat_obj = readRDS("../data/CE_CB/for_qin_briggsae_elegans/seurat_rpca_integration_joined_20240823.rds")

In [42]:
table(pData(cds)[!colnames(cds) %in% colnames(seurat_obj),]$filter)


 damaged  doublet   mutant stressed 
    7164    14746       45       19 

In [8]:
dim(seurat_obj@reductions$integrated.rpca@cell.embeddings)

[1] 403200    300

In [11]:
# Check if all cells in seurat_obj are in cds
all(colnames(seurat_obj) %in% colnames(cds))

[1] TRUE

In [ ]:
# Subset cds to only include cells in seurat_obj
cds_match = cds[,colnames(seurat_obj)]
pmeta = as.data.frame(pData(cds_match))

In [23]:
fmeta <- data.frame(gene_id = rownames(seurat_obj))
rownames(fmeta) <- rownames(seurat_obj)
seurat_obj@meta.data$filter <- NULL
adata <- AnnData(
    X =  t(seurat_obj@assays$RNA@layers$counts),
    obs = pmeta,
    var = fmeta,
)
# add obsm
for (key in names(seurat_obj@reductions)) {
    adata$obsm[[key]] <- seurat_obj@reductions[[key]]@cell.embeddings
}


In [24]:
adata$write_h5ad('../data/CE_CB/adata_cbce_1217.h5ad')

None

In [30]:
fmeta <- as.data.frame(fData(cds))
pmeta <- as.data.frame(pData(cds))
adata <- AnnData(
    X =  t(exprs(cds)),
    obs = pmeta,
    var = fmeta,
)


In [31]:
# add obsm
for (key in names(seurat_obj@reductions)) {
    adata$obsm[[key]] <- seurat_obj@reductions[[key]]@cell.embeddings[match(rownames(adata), colnames(seurat_obj)),]
}


In [39]:
adata$write_h5ad('../data/CE_CB/adata_cbce_1217_fullcds.h5ad')

None